In [1]:
a <- 1

In [2]:
a

[1] 1

In [ ]:
#### Plot || Dq x Dd (RMSSE, SPL) incl. regressions and Box-/Violin-Plots of differences: PDF 5.5 x 9.25
plotpub <- function(percentiles, Dq_fun, Dd_RMSSE_fun, Dd_SPL_fun){
  
  ## Load
  StaticWeightedSAA_RMSSE <- read_csv(paste0(PATH_RESULTS, "/StaticWeightedSAA_RMSSE.csv"))
  StaticWeightedSAA_SPL <- read_csv(paste0(PATH_RESULTS, "/StaticWeightedSAA_SPL.csv"))
  
  ## Params
  text_size <- 7.5
  model_names <- c("SWSAALOCAL", "SWSAAGLOBAL")
  model_codes <- c("A SWSAALOCAL", "B SWSAAGLOBAL")
  model_labels <- c("wSAA", "GwSAA (our model)")
  model_colors <- c("red", "blue")
  
  

  
  ## Plot Data
  getPlotData <- function(percentiles, Dq_fun, Dd_RMSSE_fun, Dd_SPL_fun){
    
    PlotData <- Results_summary %>%
      filter(IB==0.99, SL %in% c(0.5, 0.75, 0.90)) %>%
      ungroup() %>%
      select(SKU, SL, cost_SSAA, cost_SWSAALOCAL, cost_SWSAAGLOBAL, cost_ExPost) %>%
      pivot_longer(cols=c(cost_SWSAALOCAL,
                          cost_SWSAAGLOBAL),
                   names_to="model",
                   values_to="cost") %>%
      mutate(model=recode_factor(model,
                                 `cost_SWSAALOCAL`="SWSAALOCAL",
                                 `cost_SWSAAGLOBAL`="SWSAAGLOBAL"),
             Dq=eval(parse(text=Dq_fun))) %>%
      merge(Results_summary %>%
              filter(IB==0.99, SL %in% c(0.5, 0.75, 0.90)) %>%
              ungroup() %>%
              select(SKU, SL, opT_SSAA, opT_SWSAALOCAL, opT_SWSAAGLOBAL) %>%
              pivot_longer(cols=c(opT_SWSAALOCAL,
                                  opT_SWSAAGLOBAL),
                           names_to="model",
                           values_to="T_horizon_rolling") %>%
              mutate(model=recode_factor(model,
                                         `opT_SWSAALOCAL`="SWSAALOCAL",
                                         `opT_SWSAAGLOBAL`="SWSAAGLOBAL")),
            by=c("SKU", "SL", "model")) %>%
      merge(
        StaticWeightedSAA_RMSSE %>%
          select(SKU, T_horizon_rolling, RMSSE_saa, RMSSE_wsaa_local, RMSSE_wsaa_global) %>%
          pivot_longer(cols=c(RMSSE_wsaa_local, RMSSE_wsaa_global),
                       names_to="model",
                       values_to="RMSSE") %>%
          mutate(model=recode_factor(model,
                                     `RMSSE_wsaa_local`="SWSAALOCAL",
                                     `RMSSE_wsaa_global`="SWSAAGLOBAL"),
                 Dd_RMSSE=eval(parse(text=Dd_RMSSE_fun))),
        by=c("SKU", "T_horizon_rolling", "model")) %>%
      merge(
        percentiles %>%
          merge(StaticWeightedSAA_SPL %>%
                  filter(q %in% unique(percentiles$p)), 
                by.x="p", by.y="q") %>%
          select(SKU, SL, T_horizon_rolling, p, SPL_SAA, SPL_WSAA_LOCAL, SPL_WSAA_GLOBAL) %>%
          pivot_longer(cols=c(SPL_WSAA_LOCAL, SPL_WSAA_GLOBAL),
                       names_to="model",
                       values_to="SPL") %>%
          mutate(model=recode_factor(model,
                                     `SPL_WSAA_LOCAL`="SWSAALOCAL",
                                     `SPL_WSAA_GLOBAL`="SWSAAGLOBAL"),
                 Dd_SPL=eval(parse(text=Dd_SPL_fun))),
        by=c("SKU", "SL", "T_horizon_rolling", "model")) %>%
      mutate(SL=recode_factor(SL,
                              `0.5`="CR = 0.50",
                              `0.75`="CR = 0.75",
                              `0.9`="CR = 0.90"))

    
    return(PlotData)
    
  }
  PlotData <- getPlotData(percentiles, Dq_fun, Dd_RMSSE_fun, Dd_SPL_fun)
  

  

  ## Plot | CoPres x CoPred
  plotpub <- function(PlotData, Dd_name, gg_title){
    
    x_lower <- 0.2
    x_upper <- 1.6
    y_lower <- 0.2
    y_upper <- 1.4
    
    # Label for linear regression
    lm_eqn <- function(df, weighted, xlim, ylim){
      
      df <- df %>%
        filter(is.finite(x), is.finite(y),
               x >= xlim[1], x <= xlim[2],
               y >= ylim[1], y <= ylim[2]) %>%
        arrange(SL, model)
      
      result <- data.frame()
      
      for(SL_j in unique(df$SL)){
        for(model_j in unique(df$model)){
          
          data <- df %>%
            filter(SL == SL_j,
                   model == model_j)

          if(weighted){
            
            m <- lm(y ~ x, data=data, weights=weight)
            y_hat <- predict(m, data, weights=weight)
            rr <- 1 - sum(data$weight / sum(data$weight) * (data$y - y_hat)^2) / 
              sum(data$weight / sum(data$weight) * (data$y - mean(data$y))^2)
            
          } else {
            
            m <- lm(y ~ x, data=data)
            y_hat <- predict(m, data)
            rr <- 1 - sum((data$y - y_hat)^2) / 
              sum((data$y - mean(data$y))^2)
          }
          
          rr <- substitute(italic(tilde(r))*" "^2~"="~r2, 
                           list(r2 = sprintf("%.2f", rr)))

          
          res <- data.frame(SL=SL_j,
                            model=model_j,
                            a = unname(coef(m)[1]),
                            b = ifelse(!is.na(unname(coef(m)[2])), unname(coef(m)[2]), NULL),
                            rr=as.character(as.expression(rr)))
          
          result <- rbind(result, res)
          
        }
      }
      
      return(result)
      
    }
    

    # Data
    plotData <- PlotData %>%
      merge(PlotData %>%
              group_by(SKU) %>%
              summarise(cost_ExPost=mean(cost_ExPost)) %>%
              ungroup() %>%
              mutate(q_group=cut(x=cost_ExPost, 
                                 breaks=(c(-Inf,quantile(cost_ExPost, 1:10/10))),
                                 labels=c(1:10/10))) %>%
              select(SKU, q_group),
            by="SKU") %>%
      dplyr::rename(Dd=all_of(Dd_name)) %>%
      mutate(model=recode_factor(model,
                                 `SWSAALOCAL`="A SWSAALOCAL",
                                 `SWSAAGLOBAL`="B SWSAAGLOBAL"))
    

    
    # R-squared lavels
    rSquared <- lm_eqn(df=data.frame(SL=plotData$SL,
                                     model=plotData$model,
                                     x=plotData$Dd,
                                     y=plotData$Dq,
                                     weight=plotData$cost_ExPost),
                       weighted=TRUE,
                       xlim=c(-99,99),
                       ylim=c(-99,99)) %>%
      mutate(y=a+b*1.5) %>%
      select(SL, model, y) %>%
      pivot_wider(id_cols=SL, names_from=model, names_prefix="y ", values_from=y) %>%
      mutate(ynewa = ifelse(`y A SWSAALOCAL`<=`y B SWSAAGLOBAL`, 
                            `y A SWSAALOCAL` - (abs(max(abs(`y A SWSAALOCAL`-`y B SWSAAGLOBAL`),0.1)-
                                                      abs(`y A SWSAALOCAL`-`y B SWSAAGLOBAL`)))/2,
                            `y A SWSAALOCAL` + (abs(max(abs(`y A SWSAALOCAL`-`y B SWSAAGLOBAL`),0.1)-
                                                      abs(`y A SWSAALOCAL`-`y B SWSAAGLOBAL`)))/2),
             ynewb = ifelse(`y B SWSAAGLOBAL`<=`y A SWSAALOCAL`, 
                            `y B SWSAAGLOBAL` - (abs(max(abs(`y B SWSAAGLOBAL`-`y A SWSAALOCAL`),0.1)-
                                                       abs(`y B SWSAAGLOBAL`-`y A SWSAALOCAL`)))/2,
                            `y B SWSAAGLOBAL` + (abs(max(abs(`y B SWSAAGLOBAL`-`y A SWSAALOCAL`),0.1)-
                                                       abs(`y B SWSAAGLOBAL`-`y A SWSAALOCAL`)))/2)) %>%
      pivot_longer(cols=c(ynewa, ynewb),
                   names_to="model",
                   values_to="y_adj") %>%
      mutate(model=recode_factor(model,
                                 `ynewa`="A SWSAALOCAL",
                                 `ynewb`="B SWSAAGLOBAL")) %>%
      select(SL, model, y_adj) %>%
      merge(lm_eqn(df=data.frame(SL=plotData$SL,
                                 model=plotData$model,
                                 x=plotData$Dd,
                                 y=plotData$Dq,
                                 weight=plotData$cost_ExPost),
                   weighted=TRUE,
                   xlim=c(-99,99),
                   ylim=c(-99,99)) %>%
              mutate(x=x_upper*0.925),
            by=c("SL", "model"))
    
    
    
    
    # Plot
    plot_out <- plotData %>%
      ggplot() + 
      facet_grid(~SL) + 
      geom_hline(data=data.frame(yintercept=l_mid),
                 aes(yintercept=yintercept), color="black", linetype="dashed", size=0.15) +
      geom_vline(data=data.frame(xintercept=l_mid),
                 aes(xintercept=xintercept), color="black", linetype="dashed", size=0.15) +
      # Scatter
      geom_point(aes(x=Dd, y=Dq, color=model, alpha=as.numeric(q_group)), size=1) +
      # Regression
      geom_smooth(data=plotData %>%
                    select(SL, Dd, Dq, model, cost_ExPost),
                  mapping = aes(x=Dd, y=Dq, color=model, weight=cost_ExPost), method = "lm", se=F,
                  formula = y ~ x, fullrange=T, linetype="solid", size=0.75, show.legend=F) +
      # R-squared label
      geom_label(data=rSquared,
                 aes(x=x, y=y_adj, label=rr, color=model), show.legend=F, label.size=NA,
                 label.padding = unit(0.033, "lines"), label.r = unit(0.05, "lines"), 
                 fill="white", alpha=0.9, hjust=0.5, vjust=0.5, parse=TRUE, fontface="bold", size=3) +
      coord_cartesian(xlim = c(x_lower, x_upper),
                      ylim = c(y_lower, y_upper)) + 
      scale_x_continuous(name=expression("predictive performance:"~~p[k*","*tau]^xi),
                         breaks = scales::pretty_breaks(n = 9)) +
      scale_y_continuous(name=expression("prescriptive performance:"~~p[k*","*tau]^q),
                         breaks = scales::pretty_breaks(n = 9)) +
      scale_color_manual(name="Dot color intensity indicates cost impact   |   Model:",
                         values=model_colors,
                         breaks=model_codes,
                         labels=model_labels,
                         guide="none") +
      scale_fill_manual(name="",
                        values=model_colors,
                        breaks=model_codes,
                        labels=model_labels,
                        guide="none") +
      scale_alpha(name="",
                  range=c(0.1,0.5),
                  guide = "none") +
      ggtitle(label=gg_title) +
      theme_classic() + 
      theme(plot.title=element_text(face="bold", color="black", size=10, hjust=0.5),
            axis.title.x=element_text(face="bold", color="black", size=10),
            axis.text.x=element_text(color="black", size=10),
            axis.title.y=element_text(face="bold", color="black", size=10),
            axis.text.y=element_text(color="black", size=10),
            axis.line=element_line(color="black", size=0.15),
            panel.grid=element_blank(),
            strip.placement = "outside",
            strip.background=element_rect(color=NA, alpha("lightgrey", 0.75)),
            strip.text=element_text(face="bold", color="black", size=10),
            legend.title=element_text(color="black", face="bold", size=10),
            legend.text=element_text(color="black", size=10),
            legend.position="bottom",
            legend.direction="horizontal") 
    
    
    
    
    
    
    # Calculate % of plotted data given range of -1:1
    cat(paste0(gg_title,"\n"))
    for(SL in unique(plot_out$data$SL)){
      cat(paste0(SL, ": ", round(sum(plot_out$data[plot_out$data$SL==SL,]$Dd >= l_lower &
                                       plot_out$data[plot_out$data$SL==SL,]$Dd <= l_upper & 
                                       plot_out$data[plot_out$data$SL==SL,]$Dq >= l_lower & 
                                       plot_out$data[plot_out$data$SL==SL,]$Dd <= l_upper) /
                                   nrow(plot_out$data[plot_out$data$SL==SL,])*100,2),
                 "% of observations used in plot.\n"))
    }
    
    return(plot_out)
    
  }
  
  ## Dq x RMSSE
  plot_ai <- plotpub(PlotData, Dd_name="Dd_RMSSE", gg_title="Prescriptive over predictive performance (based on RMSSE)")
  
  ## Dq x SPL
  plot_bi <- plotpub(PlotData, Dd_name="Dd_SPL", gg_title="Prescriptive over predictive performance (based on SPL)")

  
  
  
  
  ## Plot | CoPres, CoPred 
  plotpub <- function(PlotData, Dd_name){
    
    
    
    
    ## Dq
    SigDq <- data.frame()
    for(CR in unique(PlotData$SL)){
      
      Dq_SWSAAGLOBAL <- (PlotData %>% subset(SL==CR & model=="SWSAAGLOBAL"))$Dq
      Dq_SWSAALOCAL <- (PlotData %>% subset(SL==CR & model=="SWSAALOCAL"))$Dq
      
      diff <- Dq_SWSAAGLOBAL - Dq_SWSAALOCAL
      
      p_value <- wilcox.test(x=Dq_SWSAAGLOBAL[is.finite(diff)],
                             y=Dq_SWSAALOCAL[is.finite(diff)],
                             alternative="two.sided",
                             paired=TRUE,
                             conf.int=TRUE,
                             exact=TRUE)$p.value
      
      SigDq <- rbind(SigDq, data.frame(SL=CR, sigDq=p_value, 
                                       median_diff=median(diff[is.finite(diff)]),
                                       shareDq=round((sum(Dq_SWSAAGLOBAL[is.finite(diff)] <= 
                                                            Dq_SWSAALOCAL[is.finite(diff)]) / 
                                                        length(Dq_SWSAAGLOBAL[is.finite(diff)]))*100,1)))
    }
    
    print(SigDq)
    

    
    
    ## Dd
    SigDd <- data.frame()
    for(CR in unique(PlotData$SL)){
      
      if(Dd_name=="Dd_RMSSE"){
        Dd_SWSAAGLOBAL <- (PlotData %>% subset(SL==CR & model=="SWSAAGLOBAL"))$Dd_RMSSE
        Dd_SWSAALOCAL <- (PlotData %>% subset(SL==CR & model=="SWSAALOCAL"))$Dd_RMSSE
      }
      
      
      if(Dd_name=="Dd_SPL"){
        Dd_SWSAAGLOBAL <- (PlotData %>% subset(SL==CR & model=="SWSAAGLOBAL"))$Dd_SPL
        Dd_SWSAALOCAL <- (PlotData %>% subset(SL==CR & model=="SWSAALOCAL"))$Dd_SPL
      }
      
      diff <- Dd_SWSAAGLOBAL - Dd_SWSAALOCAL
      
      p_value <- wilcox.test(x=Dd_SWSAAGLOBAL[is.finite(diff)],
                             y=Dd_SWSAALOCAL[is.finite(diff)],
                             alternative="two.sided",
                             paired=TRUE,
                             conf.int=TRUE,
                             exact=TRUE)$p.value
      
      SigDd <- rbind(SigDd, data.frame(SL=CR, sigDd=p_value, 
                                       median_diff=median(diff[is.finite(diff)]),
                                       shareDd=round((sum(Dd_SWSAAGLOBAL[is.finite(diff)] <= 
                                                            Dd_SWSAALOCAL[is.finite(diff)]) / 
                                                        length(Dd_SWSAAGLOBAL[is.finite(diff)]))*100,1)))
    }
    
    
    print(SigDd)
    
    
    
    
    ## Prepare plot data
    plotData <- PlotData %>%
      merge(PlotData %>%
              group_by(SKU) %>%
              summarise(cost_ExPost=mean(cost_ExPost)) %>%
              ungroup() %>%
              mutate(q_group=cut(x=cost_ExPost, 
                                 breaks=(c(-Inf,quantile(cost_ExPost, 1:10/10))),
                                 labels=c(1:10/10))) %>%
              select(SKU, q_group),
            by="SKU") %>%
      select(SKU, SL, model, Dq, all_of(Dd_name), q_group, cost_ExPost) %>%
      dplyr::rename(Dd=all_of(Dd_name)) %>%
      pivot_wider(id_cols=c(SKU, SL, q_group, cost_ExPost), 
                  names_from=model,
                  values_from=c(Dq, Dd)) %>%
      mutate(Dq_diffs = Dq_SWSAAGLOBAL - Dq_SWSAALOCAL,
             Dd_diffs = Dd_SWSAAGLOBAL - Dd_SWSAALOCAL) %>%
      pivot_longer(cols=c(Dq_diffs, Dd_diffs),
                   names_to="var",
                   values_to="diffs") 
      
    

    lim_lower <- -0.4
    lim_higher <- 0.6
    
    
    
    
    
    plot_out <- plotData %>%
      ggplot(aes(y=diffs, x=var)) +
      facet_grid(~SL) +
      geom_violin(color=NA, fill="lightgrey", alpha=0.55) +
      geom_dotplot(aes(color=diffs<=0, alpha=factor(q_group)), fill=NA,
                   binaxis='y', stackdir='center', dotsize=5, binwidth=1/460) +
      geom_point(data=data.frame(group=c("A", "B"), x=1, y=-99), 
                 aes(x=x, y=y, fill=group), shape=21, color=NA) +
      geom_boxplot(fill=NA, outlier.shape=NA, coef=0,
                   size=0.25, width=0.5) +
      geom_hline(yintercept=0, size=0.15, linetype="longdash") +
      coord_flip(ylim=c(lim_lower, lim_higher)) +
      scale_y_continuous(breaks=scales::pretty_breaks(n=6)) +
      scale_x_discrete(name="",
                       breaks=c("Dq_diffs",
                                "Dd_diffs"),
                       labels=c(expression(Delta~p[k*","*tau]^q), 
                                expression(Delta~p[k*","*tau]^xi))) +
      scale_fill_manual(name="Color intensity = cost impact     |     Upper plots:",
                        breaks=c("A", "B"),
                        values=c("blue", "red"),
                        labels=c("GwSAA (our model)", "wSAA")) +
      scale_color_manual(name=" |     Lower plots",
                         breaks=c(TRUE, FALSE),
                         values=c("blue", "red"),
                         labels=c(expression(~~Delta<=0), expression(~~Delta>0))) +
      guides(alpha=F) +
      ylab("") +
      xlab("") +
      theme_classic() + 
      theme(plot.title=element_text(face="bold", color="black", size=10, hjust=0.5),
            axis.title.x=element_blank(),
            axis.line.x=element_line(color="black", size=0.15),
            axis.text.x=element_text(color="black", size=10),
            axis.title.y=element_text(face="bold", color="black", size=10),
            axis.line.y=element_blank(),
            axis.ticks.y=element_blank(),
            axis.text.y=element_text(color="black", size=10),
            panel.grid=element_blank(),
            strip.placement = "outside",
            strip.background=element_rect(color=NA, alpha("lightgrey", 0.75)),
            strip.text=element_text(face="bold", color="black", size=10),
            legend.title=element_text(color="black", face="bold", size=10),
            legend.text=element_text(color="black", size=10),
            legend.position="bottom",
            legend.direction="horizontal") 
    
    
    
    
    
    
    
    
    
    return(plot_out)
    
    
    
      
      
      
    }
    
  ## Dq, RMSSE
  plot_aii <- plotpub(PlotData, Dd_name="Dd_RMSSE")
  
  ## Dq, SPL
  plot_bii <- plotpub(PlotData, Dd_name="Dd_SPL")
  
  
  
  
  ## Combine
  plot_a <- ggarrange(plot_ai, plot_aii,
                      ncol=1, nrow=2,
                      heights=c(1.25,1),
                      common.legend=F)
  
  plot_b <- ggarrange(plot_bi, plot_bii,
                      ncol=1, nrow=2,
                      heights=c(1.25,1),
                      common.legend=F)
  
    
   
  return(list("RMSSE" = plot_a, "SPL" = plot_b)) 
  
  
  
  
}
plot_out <- plotpub(percentiles=data.frame(SL = c(0.50, 0.75, 0.90),
                                           p = c(0.50, 0.75, 0.90)),
                    Dq_fun = "ifelse(cost==cost_SSAA,1,cost/cost_SSAA)",
                    Dd_RMSSE_fun = "ifelse(RMSSE==RMSSE_saa,1,RMSSE/RMSSE_saa)",
                    Dd_SPL_fun = "ifelse(SPL==SPL_SAA,1,SPL/SPL_SAA)")




